In [1]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("21"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Keypad Conundrum

[Open Website](https://adventofcode.com/2024/day/21)

In [2]:
import re
from collections import deque
from common.matrix import Direction, Matrix, MatrixNavigator
from functools import cache


# helper functions
class KeyPad:
    def __init__(self, input):
        self.matrix = Matrix(input)
        for x, y, value in self.matrix:
            if value == 'A':
                self.pointer = MatrixNavigator(self.matrix, x, y)
                break

    def move_to(self, value: str) -> list:
        # find end
        end = None
        for x, y, v in self.matrix:
            if v == value:
                end = (x, y)
                break

        if end is None:
            raise ValueError(f"Value {value} not found in matrix")

        start = self.pointer.get_position()
        path = find_shortest_path(self.matrix, start, end, {})
        self.pointer.set_position(end[0], end[1])

        return path

    def input_value(self, string: str) -> str:
        # find steps
        presses = []
        for next in list(string):
            path = self.move_to(next)
            for step in path:
                presses.append(direction_string(step))
            presses.append("A")

        return presses


def direction_string(direction: Direction) -> str:
    return "A" if direction is None else dir_map[direction]

direction_map = {
    Direction.UP: '^',
    Direction.DOWN: 'v',
    Direction.LEFT: '<',
    Direction.RIGHT: '>'
}
directions = direction_map.keys()

def generate_paths(matrix: Matrix, start: tuple, end: tuple) -> iter:
    queue = deque([(start, [])])
    while queue:
        current_point, path = queue.popleft()
        if current_point == end:
            yield path
            continue

        for direction in directions:
            pointer = MatrixNavigator(matrix, current_point[0], current_point[1])
            ok = pointer.move(direction)
            if ok:
                new_path = path + [direction]
                queue.append((pointer.get_position(), new_path))
    

@cache
def find_shortest_path(num_pad: Matrix, start: tuple, end: tuple, cache: dict) -> list:
    queue = deque([(start, 0, [])])
    visited = set()
    visited.add(start)

    while queue:
        # sort queue by cost
        queue = deque(sorted(queue, key=lambda x: x[1]))
        
        current_point, cost, path = queue.popleft()
        if current_point == end:
            return path

        for direction in directions:
            pointer = MatrixNavigator(matrix, current_point[0], current_point[1])
            last_direction = path[-1] if path else None
            new_cost = cost_of_direction(last_direction, direction)
            ok = pointer.move(direction)
            if ok:
                new_path = path + [direction]
                queue.append((pointer.get_position(), cost + new_cost, new_path))

    return []  # Return an empty list if no path is found


def new_number_pad() -> KeyPad:
    input = [
        ['7', '8', '9'],
        ['4', '5', '6'],
        ['1', '2', '3'],
        [' ', '0', 'A']
    ]
    return KeyPad(input)


def new_direction_pad() -> KeyPad:
    input = [
        [' ', '^', 'A'],
        ['<', 'v', '>']
    ]
    return KeyPad(input)

def get_multiplier(line: str) -> int:
    digits = re.compile(r'\d+')
    return int(digits.findall(line)[0])

def handle_line(line: str, debug: bool) -> int:
    # find first key presses
    num_pad = new_number_pad()
    result = num_pad.input_value(line)

    for i in range(0, 2):
        if debug:
            string = "".join(result)
            print(f"{string} ({len(string)})")
        dir_pad = new_direction_pad()
        result = dir_pad.input_value(result)

    total = get_multiplier(line) * len(result)
    if debug:
        print(f'line: {line}')
        print(f'result: {"".join(result)}')
        print(f'length: {len(result)}')
        print(f'total: {total}')

    return total


# test case (part 1)
def part_1(inputs: list, debug: bool) -> int:
    total = 0
    for input in inputs:
        total += handle_line(input, debug)
        if debug:
            print()
    return total


inputs = ["029A","980A","179A","456A","379A"]
answers = {
    "029A": "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A",
    "980A": "<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A",
    "179A": "<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
    "456A": "<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A",
    "379A": "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
}

for input in inputs:
    answer_inputs = answers[input]
    multiplier = get_multiplier(input)
    answer = len(answer_inputs) * multiplier
    
    result = part_1([input], True)
    display(result)
    if result != answer:
        print(f"Expected {answer} but got {result} for input {input}")
        
    assert result == answer

result = part_1(inputs, True)
display(result)
assert result == 126384

<A^A^^>AvvvA (12)
v<<A>>^A<A>A<AA>vA^Av<AAA^>A (28)
line: 029A
result: v<A<AA>>^AvAA^<A>Av<<A>>^AvA^Av<<A>>^AAvA<A^>A<A>Av<A<A>>^AAA<A>vA^A
length: 68
total: 1972



1972

^^^A<AvvvA>A (12)
<AAA>Av<<A>>^Av<AAA^>AvA^A (26)
line: 980A
result: v<<A>>^AAAvA^Av<A<AA>>^AvAA^<A>Av<A<A>>^AAA<A>vA^Av<A^>A<A>A
length: 60
total: 58800



58800

^<<A^^A>>AvvvA (14)
<Av<AA>>^A<AA>AvAA^Av<AAA^>A (28)
line: 179A
result: v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AAvA^Av<A^>AA<A>Av<A<A>>^AAA<A>vA^A
length: 68
total: 12172



...

29184

^A^^<<A>>AvvvA (14)
<A>A<AAv<AA>>^AvAA^Av<AAA^>A (28)
line: 379A
result: v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<A>vA^A
length: 68
total: 25772



25772

Expected 24256 but got 25772 for input 379A
<A>Av<<AA>^AA>AvAA^A<vAAA>^A (28)
^A<<^^A>>AvvvA (14)


AssertionError: 

In [9]:
# real case (part 1)
result = part_1(puzzle.input().lines_as_str(), False)
display(result)

188078

In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        display(lines)
    return 0


result = part_2(puzzle.example(0), True)
display(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input().lines_as_str(), False)
display(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()